<h1>Train data from Kaggle Twitter sentiment (Sentiment140 dataset with 1.6 million tweets)</h1>
https://www.kaggle.com/kazanova/sentiment140

In [ ]:
# Import library


pip install contractions

from google.colab import drive
drive.mount('/content/drive')


import pandas as pd

import re

import numpy as np

#for data cleansing
from nltk.stem.snowball import SnowballStemmer

import contractions

from sklearn.model_selection import train_test_split


Mounted at /content/drive


# Clean Data from Kaggle

In [ ]:
df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/training.1600000.processed.noemoticonf.csv',encoding ='latin',engine='python')

In [ ]:
df.shape

(1600000, 2)

In [ ]:
# checking  if any null value

df.isnull().sum()

sentiment     0
tweet_text    0
dtype: int64

In [ ]:
# 0 is negative, 1 is positive >> replace 4 with 1
df['sentiment'] = df['sentiment'].apply(lambda x: 1 if x==4 else x)

In [ ]:
# Check if the sample is balanace or not 
#dataset are with target 0 on the first 80000 rows and target 1 on the last 80000 rows

df['sentiment'].value_counts()

1    800000
0    800000
Name: sentiment, dtype: int64

In [ ]:
#shuffle the dataframe
newdf = df.sample(frac=1).reset_index(drop=True)

In [ ]:
# Data clensing function

def cleantext(text):

    text = text.lower().strip()
    text = re.sub(r'@[A-za-z0-9]+','',text) # remove @mentions
    text = re.sub(r'RT[\s]+','',text) #remove retweet
    text = re.sub(r'https?:\/\/\S+','',text) #remove hyperlink
    text = re.sub(r'#','',text) #remove #

    text = text.replace('\n',' ')

    text = contractions.fix(text)

    stemmer = SnowballStemmer(language='english')
    text = stemmer.stem(text)
    text = re.sub("[\.\,\!\?\:\;\-\=\*\"\'\“\(\)\_]", "",text) # remove puntucation
    text = re.sub("[0123456789]", "", text) # remove number

    text = text.replace('  ',' ')

    return text.strip()


# Apply clean text function

newdf['tweet_text'] = newdf['tweet_text'].apply(cleantext)


# Start building the model with tensorflow

In [ ]:
from tensorflow.keras import Model, Sequential
from tensorflow.keras.layers import Activation, Dense, Dot, Embedding, Flatten, GlobalAveragePooling1D, Reshape
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from keras.preprocessing.text import Tokenizer
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.sequence import pad_sequences

In [ ]:

# Apply Train test split

X = newdf['tweet_text'].apply(str)
y = newdf['sentiment']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=20)



1120000
480000


In [ ]:
#Tokenize and fit with training data
from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
word_index = tokenizer.word_index

In [ ]:
from keras.preprocessing.sequence import pad_sequences

X_train = pad_sequences(tokenizer.texts_to_sequences(X_train),maxlen = 200)
X_test = pad_sequences(tokenizer.texts_to_sequences(X_test),maxlen = 200)

<h2>Load pre-trained word embeddings -  GloVe embeddings </h2>

In [ ]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip -q glove.6B.zip

--2020-11-16 16:54:26--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2020-11-16 16:54:27--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2020-11-16 16:54:28--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

In [ ]:
path_to_glove_file = "/content/drive/My Drive/Colab Notebooks/glove.6B.200d.txt"

embeddings_index = {}
with open(path_to_glove_file) as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs


In [ ]:
from tensorflow.keras.layers import Embedding

embedding_layer = Embedding(
    vocab_len,
    embedding_dim,
    embeddings_initializer=keras.initializers.Constant(embedding_matrix),
    trainable=False,
)

In [ ]:
# from tensorflow.keras.layers import SpatialDropout1D
from tensorflow.keras.callbacks import ModelCheckpoint

In [ ]:
from tensorflow.keras import layers

int_sequences_input = layers.Input(shape=(200,), dtype='int32')
embedding_sequences = embedding_layer(int_sequences_input)
x = layers.Conv1D(128, 5, activation="relu")(embedding_sequences)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(128, 5, activation='relu')(x)
x = layers.Bidirectional(layers.LSTM(64, dropout=0.2, recurrent_dropout=0.2))(x)
x = layers.Dense(128, activation='relu')(x)
x = layers.Dropout(0.5)(x)
preds = layers.Dense(1, activation="sigmoid")(x)
model = tf.keras.Model(int_sequences_input, preds)
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 200)]             0         
_________________________________________________________________
embedding (Embedding)        (None, 200, 200)          64411600  
_________________________________________________________________
conv1d (Conv1D)              (None, 196, 128)          128128    
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 39, 128)           0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 35, 128)           82048     
_________________________________________________________________
bidirectional (Bidirectional (None, 128)               98816     
_________________________________________________________________
dense (Dense)                (None, 128)              

In [ ]:
from tensorflow.keras.optimizers import Adam

model.compile(optimizer="adam", loss='binary_crossentropy',
              metrics=['accuracy'])

my_callbacks = [tf.keras.callbacks.EarlyStopping(patience=3, 
                                                 monitor = 'val_loss'),]
model.fit(X_train, y_train, batch_size=128, epochs=20, 
          validation_data=(X_test,y_test),callbacks=my_callbacks)

Epoch 1/20
8750/8750 [==============================] - 1256s 143ms/step - loss: 0.4957 - accuracy: 0.7583 - val_loss: 0.4716 - val_accuracy: 0.7734
Epoch 2/20
8750/8750 [==============================] - 1245s 142ms/step - loss: 0.4634 - accuracy: 0.7794 - val_loss: 0.4630 - val_accuracy: 0.7786
Epoch 3/20
8750/8750 [==============================] - 1246s 142ms/step - loss: 0.4494 - accuracy: 0.7878 - val_loss: 0.4625 - val_accuracy: 0.7797
Epoch 4/20
8750/8750 [==============================] - 1250s 143ms/step - loss: 0.4390 - accuracy: 0.7945 - val_loss: 0.4653 - val_accuracy: 0.7797
Epoch 5/20
8750/8750 [==============================] - 1264s 144ms/step - loss: 0.4304 - accuracy: 0.7991 - val_loss: 0.4640 - val_accuracy: 0.7800
Epoch 6/20
8750/8750 [==============================] - 1246s 142ms/step - loss: 0.4230 - accuracy: 0.8039 - val_loss: 0.4602 - val_accuracy: 0.7810
Epoch 7/20
8750/8750 [==============================] - 1248s 143ms/step - loss: 0.4165 - accuracy: 0.8078

In [ ]:
#save the model
model.save("/content/drive/My Drive/Colab Notebooks/model/twitter_sentiment")

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: /content/drive/My Drive/Colab Notebooks/model/twitter_sentiment/assets


In [ ]:
#pickle and save the tokenizer
import pickle
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)